In [1]:
import sys
import os

def is_colab_env():
    return "google.colab" in sys.modules

def mount_google_drive(drive_dir="/content/drive/", repo_dir="MyDrive/repositories/deepfake-detection"):
    # mount google drive
    from google.colab import drive
    drive.mount(drive_dir)

    # change to correct working directory
    import os
    os.chdir(f"{drive_dir}{repo_dir}")
    print(os.listdir()) # verify content

def resolve_path(levels_deep=3):
    if is_colab_env():
        mount_google_drive()
    else:
        # Get the directory of the current script
        current_dir = os.path.dirname(os.path.abspath('__file__'))

        # Construct the path to the parent directory
        for i in range(levels_deep):
            current_dir = os.path.dirname(current_dir)

        # Add the parent directory to sys.path
        sys.path.append(current_dir)
        print(sys.path)

resolve_path(2)

['c:\\Users\\jinxy\\anaconda3\\envs\\df-env\\python310.zip', 'c:\\Users\\jinxy\\anaconda3\\envs\\df-env\\DLLs', 'c:\\Users\\jinxy\\anaconda3\\envs\\df-env\\lib', 'c:\\Users\\jinxy\\anaconda3\\envs\\df-env', '', 'c:\\Users\\jinxy\\anaconda3\\envs\\df-env\\lib\\site-packages', 'c:\\Users\\jinxy\\anaconda3\\envs\\df-env\\lib\\site-packages\\win32', 'c:\\Users\\jinxy\\anaconda3\\envs\\df-env\\lib\\site-packages\\win32\\lib', 'c:\\Users\\jinxy\\anaconda3\\envs\\df-env\\lib\\site-packages\\Pythonwin', 'g:\\My Drive\\repositories\\deepfake-detection']


In [2]:
# import local config
import config

In [3]:
# from src.adapters.datasets.wilddeepfake import load_streaming_dataset
from datasets import load_dataset

In [4]:
from src.transforms.frequency import fft_real_imag

In [ ]:
dataset = load_dataset("xingjunm/WildDeepfake", streaming=True, split="train")

Resolving data files:   0%|          | 0/963 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/157 [00:00<?, ?it/s]

In [6]:
dataset

IterableDataset({
    features: ['png', '__key__', '__url__'],
    num_shards: 963
})

In [ ]:
import numpy as np

real_fft = []
fake_fft = []
count = 0
for d in dataset:
    img = d['png']
    fft_result = fft_real_imag(img)  # Should return a numpy array
    if "real" in d["__key__"]:
        real_fft.append(fft_result)
    else:
        fake_fft.append(fft_result)


# Convert lists to arrays
real_fft = np.stack(real_fft)
fake_fft = np.stack(fake_fft)

# Compute mean FFT (element-wise mean)
mean_real_fft = np.mean(real_fft, axis=0)
mean_fake_fft = np.mean(fake_fft, axis=0)


MemoryError: 

In [ ]:
import matplotlib.pyplot as plt

# mean_real_fft and mean_fake_fft have shape (2, H, W)
mean_real_real = mean_real_fft[0]      # Real part for real images
mean_real_imag = mean_real_fft[1]      # Imaginary part for real images
mean_fake_real = mean_fake_fft[0]      # Real part for fake images
mean_fake_imag = mean_fake_fft[1]      # Imaginary part for fake images

fig, axs = plt.subplots(2, 2, figsize=(10, 8))
axs[0, 0].imshow(mean_real_real, cmap='gray')
axs[0, 0].set_title('Real Images - Real Part')
axs[0, 1].imshow(mean_real_imag, cmap='gray')
axs[0, 1].set_title('Real Images - Imaginary Part')
axs[1, 0].imshow(mean_fake_real, cmap='gray')
axs[1, 0].set_title('Fake Images - Real Part')
axs[1, 1].imshow(mean_fake_imag, cmap='gray')
axs[1, 1].set_title('Fake Images - Imaginary Part')
for ax in axs.ravel():
    ax.axis('off')
plt.tight_layout()
plt.show()